
#  Visualizing the Taxes and Transfers System

## How to create a plot

To help you understand how GETTSIM works internally and how you are able to implement
custom reforms, you can visualize the tax and transfer system. This tutorial explains
how to create a graphic and what information you can get from it. It also explains
GETTSIM’s design to some extent.

In [ ]:
from gettsim import plot_dag, set_up_policy_environment

For the visualization, we need to set up our policy environment.

In [ ]:
environment = set_up_policy_environment(date=2020)

Functions inside GETTSIM are a little bit special. Take for example `abgeltungssteuer__betrag_y_sn`
which is documented
[here](https://gettsim.readthedocs.io/en/stable/gettsim_objects/functions.html#gettsim.functions.abgeltungssteuer__betrag_y_sn).
The signature of the function is

```python
def abgeltungssteuer__betrag_y_sn(einkommensteuer__einkünfte__aus_kapitalvermögen__betrag_y_sn, abgelt_st_params):
    pass
```

This functions has two arguments and one of them passes parameters to the function. Most
functions require some parameters, but it is not necessary. The names of the arguments
correspond to either a variable in the data provided by the user or to another function
which, in turn, also relies on some arguments.

Here, ``abgelt_st_params`` is a paramter file which includes paramters needed to
calculate the capital income tax. ``abgeltungssteuer__betrag_y_sn`` on the other hand is
itself a function which is documented
[here](https://gettsim.readthedocs.io/en/stable/gettsim_objects/functions.html#gettsim.functions.zu_verst_kapital_eink_sn).
By using ``einkommensteuer__einkünfte__aus_kapitalvermögen__betrag_y_sn`` as an argument name, GETTSIM knows to pass the
data computed by the function ``einkommensteuer__einkünfte__aus_kapitalvermögen__betrag_y_sn`` to ``abgeltungssteuer__betrag_y_sn``.

This dependency relationship can be analyzed for all functions passed to GETTSIM and be
visualized in a dag. Below you can see a plot of all variables which are directly
connected to ``abgeltungssteuer__betrag_y_sn``. The arrows point from dependencies to
dependents. Each node is either a functions or a variable.

In [ ]:
plot_dag(
    environment=environment,
    selectors=[
        {
            "node": "abgeltungssteuer__betrag_y_sn",
            "type": "neighbors",
        }
    ],
).show()

The general interface of the plotting function is similar to ``compute_taxes_and_transfers()``, but without the ``data`` and ``params`` argument. Here is the complete signature.

In [ ]:
plot_dag?

In the following, you see many ways to either select different subsets of the graph or style the plot.

## Orientation

By default orientation of graph is vertical. It is possible to change the orientation of graph to horizontal one by setting `plot_dag(..., orientation='h') `

In [ ]:
plot_dag(
    environment=environment,
    selectors=[
        {
            "node": "abgeltungssteuer__betrag_y_sn",
            "type": "neighbors",
        }
    ],
    orientation="h",
).show()

## Labels

By default (when `show_labels` is set to its default of `None`), all all names are displayed next to the node whenever the number of nodes is smaller than 10. For 10 nodes or more like in the example below, the names are by default displayed next to the node only when hovering over it.

In [ ]:
plot_dag(
    environment=environment,
    selectors=[
        {"node": "sozialversicherung__geringfügig_beschäftigt", "type": "descendants"}
    ],
    orientation="h",
).show()

`show_labels` can be also set to `True` or `False` to enfore displaying or not displaying all labels.

In [ ]:
selector = {
    "type": "descendants",
    "node": "sozialversicherung__geringfügig_beschäftigt",
}
plot_dag(
    environment=environment, show_labels=False, selectors=selector, orientation="h"
).show()

## Hover info (source code)

It is also possible to address the source code of the function. By setting `plot_dag(..., hover_source_code=True)`

In [ ]:
plot_dag(
    environment=environment,
    selectors=[
        {
            "node": "abgeltungssteuer__betrag_y_sn",
            "type": "neighbors",
        }
    ],
    orientation="h",
    hover_source_code=True,
).show()

## Selectors

Selectors allow you to visualize only a subset of the complete graph of the tax and transfer systems. They can be passed to the `selectors` argument of the `plot_dag()` function. There exist some ways to define a selector and they can be combined with one another. Let us discuss each selector on its own first.

## Basics

It is always possible to pass a string or a list of strings to `selectors`. In this case only the given nodes are displayed in the plot.

In [ ]:
selectors = "einkommen__kapitaleinnahmen_y_sn"

plot_dag(environment=environment, selectors=selectors, orientation="h").show()

Using a list of variable names, we can select multiple nodes.

In [ ]:
selectors = [
    "einkommen__kapitaleinnahmen_y_sn",
    "abgeltungssteuer__betrag_y_sn",
]

plot_dag(environment=environment, selectors=selectors, orientation="h").show()

Passing a string or a list of strings to `selectors` is actually a shortcut for the richer interface for selecting nodes. Selectors are usually represented as dictionaries. The corresponding dictionary for selecting a list of nodes is

In [ ]:
selector = {
    "type": "nodes",
    "node": [
        "einkommen__kapitaleinnahmen_y_sn",
        "abgeltungssteuer__betrag_y_sn",
    ],
    "select": True,  # optional
}

Let us go through the keys of the dictionary one by one.

1. `"type"` specifies the type of the selector. For a single node or a list of nodes the type is `"nodes"`.
2. `"node"` always refers to the node or nodes to which the selector is applied. In this case, it is the list of node names.
3. `"select"` specifies whether the nodes should be selected or de-selected. If you do not specify `"select"` it is assumed to be `True`.Let us go through the keys of the dictionary one by one.


### De-selecting Nodes

It is also possible to specify selectors which de-select some nodes. Note that,

- De-selectors are applied after nodes have been selected.
- If no selectors are provided, de-selectors de-select nodes from the complete DAG.
- Selection and de-selection works for all selector types which follow.

For a simple and silly example, we want to reproduce the graph with the single node for `einkommen__kapitaleinnahmen_y_sn` after, but starting from the last plot which also showed `abgeltungssteuer__betrag_y_sn`.

First, we define the selectors. The first selector or dictionary in the list selects the two nodes. Note that the `"select"` key is `True` by default. The second key in the de-selects `"abgeltungssteuer__betrag_y_sn"`.

In [ ]:
selectors = [
    {
        "type": "nodes",
        "node": [
            "einkommen__kapitaleinnahmen_y_sn",
            "abgeltungssteuer__betrag_y_sn",
        ],
    },
    {
        "type": "nodes",
        "node": "abgeltungssteuer__betrag_y_sn",
        "select": False,
    },
]
plot_dag(environment=environment, selectors=selectors, orientation="v").show()

### Ancestors and Descendants

Two other types of selectors allow you to pick one node and all nodes which appear before or after this node. We call the nodes ancestors or descendants, respectively. To select `"abgeltungssteuer__betrag_y_sn"` which is the calculated taxable capital income per Steuernummer and all its ancestors, do the following.

In [ ]:
selector = {
    "type": "ancestors",
    "node": "abgeltungssteuer__betrag_y_sn",
}
plot_dag(environment=environment, selectors=selector, orientation="h").show()

To see the variables which are explicitly and implicitly dependent on the information in `"sozialversicherung__geringfügig_beschäftigt"` use the type `"descendants"`.

In [ ]:
selector = {
    "type": "descendants",
    "node": "sozialversicherung__geringfügig_beschäftigt",
}
plot_dag(environment=environment, selectors=selector, orientation="h").show()

### Neighbors

Another common way to look at a graph is to visualize a node and its neighbors, its ancestors and descendants. Let us take a look at `"einkommen__kapitaleinnahmen_y_sn"` again and visualize its direct neighbors.

In [ ]:
selector = {"type": "neighbors", "node": "einkommen__kapitaleinnahmen_y_sn"}
plot_dag(environment=environment, selectors=selector, orientation="h").show()

It is also possible to look at more distant neighbors or neighbors of order 2, 3, ... . This can be done by the `"order"` key which is 1 by default.

In [ ]:
selector = {"type": "neighbors", "node": "einkommen__kapitaleinnahmen_y_sn", "order": 2}
plot_dag(environment=environment, selectors=selector, orientation="h").show()